<hr style="border:1px solid gray"> </hr>

# Monodromy 

*Documentation*: https://www.juliahomotopycontinuation.org/HomotopyContinuation.jl/stable/monodromy/

An alternative to using the `solve` function is solving a polynomial system $F=(f_1,\ldots,f_n)$ by monodromy. 

Here is the basic idea:


Suppose $x$ is a solution $F(x)=0$ and that $F(x)=F(x,u)$ is a point in a family of polynomial systems 

$$P=\{F(x,u) \mid u\in \mathbb{C}^k\}.$$

The monodromy method consists in moving around $u$ in a loop starting and ending at $u_0$ while tracking $x$ along that loop. After one iteration usually one has found a new solution $y\in \mathbb{C}^n$. This process then is repeated until some stopping criterion is fulfilled.


Here is a simple example: take

$$F(x,y,u_1,u_2) = \begin{bmatrix} x^4 + y - 2u_1\\\ x^4 + x^2 - 2u_2y^2 \end{bmatrix}.$$

For the initial parameters $u_0=(1,1)$ we have the solution $(x,y) = (1,1)$. For finding all solutions of $F(x,y,1,1)$ we use

In [1]:
using HomotopyContinuation
@var x y 
@var u[1:2] # u are the parameters
F = System([x^4 + y - 2u[1], x^4 + x^2 - 2*u[2]*y^2],
            variables = [x;y],
            parameters = u)

System of length 2
 2 variables: x, y
 2 parameters: u₁, u₂

 -2*u₁ + y + x^4
 -2*y^2*u₂ + x^2 + x^4

In [2]:
initial_solutions = [[1; 1]]
initial_parameter = [1, 1]
monodromy_solve(F, initial_solutions, initial_parameter)

Solutions found: 8 	 Time: 0:00:02
  tracked loops (queued):            96 (0)
  solutions in current (last) loop:  0 (0)
  generated loops (no change):       12 (5)


MonodromyResult
• return_code → :heuristic_stop
• 8 solutions
• 96 tracked loops
• random_seed → 0x3eefcad6

It is possible to not pass initial solution and initial parameters. 

In this case, the software will generate random complex parameters and try to compute one solutions with Newton's method.

In [3]:
S = monodromy_solve(F)

MonodromyResult
• return_code → :heuristic_stop
• 8 solutions
• 72 tracked loops
• random_seed → 0xdb4ef88e

In [4]:
parameters(S)

2-element Vector{ComplexF64}:
 -0.15997161328947587 + 0.3942054554810579im
  0.23224876736623443 - 0.11205578552447898im

<hr style="border:1px solid gray"> </hr>

# Group Actions

If the set of solutions of `F` is invariant under some group actions you can exploit this in your computation.


In the above example, the group that interchanges `x` and `y` acts on the solution set of `F`. We can use the group that multiplies `x` by $\pm 1$.

In [5]:
G = GroupActions( a -> ([-a[1], a[2]], ))
monodromy_solve(F, 
                initial_solutions, 
                initial_parameter, 
                group_actions = G)

MonodromyResult
• return_code → :heuristic_stop
• 4 classes of solutions (modulo group action)
• 32 tracked loops
• random_seed → 0x202149d5

Now, we found only 4 solutions: one from each orbit. If we suppress computing with equivalence classes, then

In [6]:
monodromy_solve(F, 
                initial_solutions, 
                initial_parameter, 
                group_actions = G, 
                equivalence_classes = false)

MonodromyResult
• return_code → :heuristic_stop
• 8 solutions
• 56 tracked loops
• random_seed → 0x49cb810a

This also works:

In [7]:
monodromy_solve(F, group_actions = G)

MonodromyResult
• return_code → :heuristic_stop
• 2 classes of solutions (modulo group action)
• 12 tracked loops
• random_seed → 0x06296412

<hr style="border:1px solid gray"> </hr>

# Permutations

Monodromy loops incude a permutations on the zeros of $F=0$. 

They can be extracted as follows:

In [19]:
S = monodromy_solve(F; permutations = true) # the default is `permutations = false`

MonodromyResult
• return_code → :heuristic_stop
• 8 solutions
• 72 tracked loops
• random_seed → 0x7ac03e60

In [20]:
permutations(S)

8×5 Matrix{Int64}:
 1  2  1  1  1
 2  3  5  4  7
 3  4  3  3  3
 4  1  6  2  8
 5  7  2  8  5
 6  8  4  7  6
 7  6  7  5  2
 8  5  8  6  4

This table shows the permuted indices in each column. The column represent different loops

In [22]:
permutations(S; reduced = false)

8×9 Matrix{Int64}:
 1  1  2  1  1  1  1  1  1
 2  2  3  5  4  2  7  5  2
 3  3  4  3  3  3  3  3  3
 4  4  1  6  2  4  8  6  4
 5  5  7  2  8  5  5  2  5
 6  6  8  4  7  6  6  4  6
 7  7  6  7  5  7  2  7  7
 8  8  5  8  6  8  4  8  8